- 封包數量
- 傳輸流量
- 封包流數量
- 封包流平均流量
- 封包流數量 / 平均單位封包傳輸流量
- 連線節點數

In [2]:
import pandas as pd
from pathlib import Path
from glob import glob
import os
from datetime import datetime, timedelta
import warnings
import gc

interval = 30 # seconds

In [2]:
df = pd.read_csv('netflow/200702111400/256500/256507-212_136_45_198.csv')
df[["ts", "te"]] = df[["ts", "te"]].astype(dtype='datetime64[ns]')
feature = pd.DataFrame()
feature["timeStart"] = df["ts"]
feature["timeStop"] = df["te"]
feature["duration"] = df["td"]
feature["protocol"] = df["pr"]
feature["srcIP"] = df["sa"]
feature["packets"] = df["ipkt"] + df["opkt"]
feature["bytes"] = df["ibyt"] + df["obyt"]
feature["flows"] = 1
feature["packsets/flows"] = feature["packets"] / feature["flows"]
feature["bytes/flows"] = feature["bytes"] / feature["flows"]
feature["bytes/packets"] = feature["bytes"] / feature["packets"]
feature["flows/(bytes/packets)"] = feature["flows"] / feature["bytes/packets"]
feature["nodes"] = 1
feature.to_csv("netflow/200702111400/256500/256507-212_136_45_198.csv", index=False)

In [3]:
for filename in glob(Path('netflow/200702111400/*/*').__str__()):
    try:
        targetFilename = filename.replace("netflow", "feature")
        df = pd.read_csv(filename)
        df[["ts", "te"]] = df[["ts", "te"]].astype(dtype='datetime64[ns]')
        feature = pd.DataFrame()
        feature["timeStart"] = df["ts"]
        feature["timeStop"] = df["te"]
        feature["duration"] = df["td"]
        feature["protocol"] = df["pr"]
        feature["srcIP"] = df["sa"]
        feature["packets"] = df["ipkt"] + df["opkt"]
        feature["bytes"] = df["ibyt"] + df["obyt"]
        feature["flows"] = 1
        feature["packsets/flows"] = feature["packets"] / feature["flows"]
        feature["bytes/flows"] = feature["bytes"] / feature["flows"]
        feature["bytes/packets"] = feature["bytes"] / feature["packets"]
        feature["flows/(bytes/packets)"] = feature["flows"] / feature["bytes/packets"]
        feature["nodes"] = 1
        os.makedirs(os.path.dirname(targetFilename), exist_ok=True)
        feature.to_csv(targetFilename, index=False)
    except:
        print("Error: " + filename)

Error: netflow\200702111400\0\1-avg_bps.csv


In [3]:
for filename in glob(Path('feature/200702111400/*/*').__str__()):
    try:
        targetFilename = filename.replace("feature", f"interval{interval}")
        if os.path.exists(targetFilename):
            continue
        df = pd.read_csv(filename)
        df[["timeStart", "timeStop"]] = df[["timeStart", "timeStop"]].astype(dtype='datetime64[ns]')
        start_time = datetime.fromisoformat("2007-02-11T13:00:00")
        end_time = start_time + timedelta(minutes=15)
        result = []
        while start_time <= end_time:
            filtered = df[(df["timeStart"] >= start_time) & (df["timeStart"] < start_time + timedelta(seconds=interval))]
            result.append(
                {
                    "timeStart": start_time,
                    "SrcIP": df["srcIP"].unique().tolist()[0],
                    "packets": filtered["packets"].sum(),
                    "bytes": filtered["bytes"].sum(),
                    "flows": filtered["flows"].sum(),
                    "packets/flows": filtered["packets"].sum() / filtered["flows"].sum() if filtered["flows"].sum() != 0 else 0,
                    "bytes/flows": filtered["bytes"].sum() / filtered["flows"].sum() if filtered["flows"].sum() != 0 else 0,
                    "bytes/packets": filtered["bytes"].sum() / filtered["packets"].sum() if filtered["packets"].sum() != 0 else 0,
                    "flows/(bytes/packets)": filtered["flows"].sum() / (filtered["bytes"].sum() / filtered["packets"].sum()) if filtered["bytes"].sum() != 0 and filtered["packets"].sum() != 0 else 0,
                    "nodes": filtered["nodes"].sum(),
                }
            )
            start_time += timedelta(seconds=interval)
        result = pd.DataFrame(result)
        # print(result)
        # break
        os.makedirs(os.path.dirname(targetFilename), exist_ok=True)
        result.to_csv(targetFilename, index=False)
        del df, result, filtered, start_time, end_time, targetFilename, filename
        gc.collect()
    except Exception as e:
        print("Error: " + filename)
        print(e)